In [1]:
import pandas as pd
import numpy as np
import datetime as dt


In [2]:
# er moeten minimaal 2 minuten zitten tussen de zelfde persoon op de zelfde dag op de zelfde locatie anders 
# zien we dit als een registratie fout

delta=dt.timedelta(minutes = 2)

In [3]:
file= "d:/locatusdata_bewerkt.csv"
usecols=[1,2,3,4]
df1=pd.read_csv(file, sep=',', usecols=usecols, nrows=10000)
df1.columns = ['Locatie', 'DatumTijd','Duur','Persoon']

In [4]:
df1['DatumTijd'] =  pd.to_datetime(df1['DatumTijd'], format='%Y-%m-%d %H:%M:%S')

In [5]:
df1['Jaar'] =  df1['DatumTijd'].dt.year
df1['Maand'] =  df1['DatumTijd'].dt.month
df1['Dag'] =  df1['DatumTijd'].dt.day
df1['Uur'] =  df1['DatumTijd'].dt.hour
df1['Minuut'] =  df1['DatumTijd'].dt.minute
df1['Seconde'] =  df1['DatumTijd'].dt.second
df1['Dagnaam'] =  df1['DatumTijd'].dt.weekday_name
df1['Datum'] =  df1['DatumTijd'].dt.date
df1['Tijd'] =  df1['DatumTijd'].dt.time

In [6]:
df1['Datum'] =  pd.to_datetime(df1['Datum'], format='%Y-%m-%d')

In [7]:
df1.dtypes

Locatie               int64
DatumTijd    datetime64[ns]
Duur                float64
Persoon               int64
Jaar                  int64
Maand                 int64
Dag                   int64
Uur                   int64
Minuut                int64
Seconde               int64
Dagnaam              object
Datum        datetime64[ns]
Tijd                 object
dtype: object

In [8]:
df1=df1.sort_values(['Persoon', 'Datum','Tijd'], ascending=[True, True,True])

In [9]:
df1.shape

(10000, 13)

In [10]:
def Verwijder_Foute_Tellingen(df):
    if len(df)>1:
        df=df[df['DatumTijd'].shift(-1)-df['DatumTijd']>delta]
    return df

In [11]:
df1=df1.groupby(['Persoon','Datum','Locatie']).apply(Verwijder_Foute_Tellingen)

In [12]:
df1.shape

(9834, 13)

In [13]:
df1=df1.reset_index(drop=True)

In [14]:
df1.head()

,Locatie,DatumTijd,Duur,Persoon,Jaar,Maand,Dag,Uur,Minuut,Seconde,Dagnaam,Datum,Tijd
0,2054,2017-10-12 20:28:04,0.0,306,2017,10,12,20,28,4,Thursday,2017-10-12,20:28:04
1,2054,2017-09-29 13:47:24,601.0,608,2017,9,29,13,47,24,Friday,2017-09-29,13:47:24
2,2054,2017-11-12 15:21:50,NaN,608,2017,11,12,15,21,50,Sunday,2017-11-12,15:21:50
3,2054,2017-06-16 20:00:32,0.0,643,2017,6,16,20,0,32,Friday,2017-06-16,20:00:32
4,2054,2016-10-27 16:46:21,0.0,895,2016,10,27,16,46,21,Thursday,2016-10-27,16:46:21


In [15]:
def Bereken_Aantal_Registraties_Per_Dag_En_Route(df):
    df['Aantal_Registraties_Per_Dag']=len(df)
    df['Locatie_Naar']=df['Locatie'].shift(-1)
    df['Eind_tijd']=df['Tijd'].shift(-1)
    df['Route_Naar']=df['Locatie'].astype(str)+'-'+df['Locatie'].shift(-1).astype(str)
    df['Route_Totaal'] ='-'.join(str(e) for e in np.array(df['Locatie']))
    df['Route_Stapnummer'] =  range(1, 1 + len(df))
    return df


In [16]:
df1=df1.groupby(['Persoon','Datum']).apply(Bereken_Aantal_Registraties_Per_Dag_En_Route)

In [66]:
df1[df1['Aantal_Registraties_Per_Dag']>2].head()

,Locatie,DatumTijd,Duur,Persoon,Jaar,Maand,Dag,Uur,Minuut,Seconde,Dagnaam,Datum,Tijd,Aantal_Registraties_Per_Dag,Locatie_Naar,Eind_tijd,Route_Naar,Route_Totaal,Route_Stapnummer,test
4979,2054,2016-12-15 10:14:41,NaN,905391,2016,12,15,10,14,41,Thursday,2016-12-15,10:14:41,4,2054.0,10:22:41,2054-2054.0,2054-2054-2054-2054,1,0.0
4980,2054,2016-12-15 10:22:41,NaN,905391,2016,12,15,10,22,41,Thursday,2016-12-15,10:22:41,4,2054.0,16:18:41,2054-2054.0,2054-2054-2054-2054,2,NaN
4981,2054,2016-12-15 16:18:41,NaN,905391,2016,12,15,16,18,41,Thursday,2016-12-15,16:18:41,4,2054.0,18:16:41,2054-2054.0,2054-2054-2054-2054,3,NaN
4982,2054,2016-12-15 18:16:41,NaN,905391,2016,12,15,18,16,41,Thursday,2016-12-15,18:16:41,4,NaN,NaN,2054-nan,2054-2054-2054-2054,4,NaN
5028,2054,2017-05-11 10:17:33,NaN,905391,2017,5,11,10,17,33,Thursday,2017-05-11,10:17:33,3,2054.0,13:57:32,2054-2054.0,2054-2054-2054,1,0.0


In [ ]:
def Bereken_Aantal_Dagen_Achter_Elkaar(df):
    if len(df) >1:
         print (df['Datum'].agg('count',df['Datum']))
         df['Aantal_Dagen_Achter_Elkaar'] = (df.Datum.shift(1) == (df.Datum + pd.DateOffset(days=1))).astype(int).cumsum()
    return df

In [ ]:
df1=df1.groupby(['Persoon']).apply(Bereken_Aantal_Dagen_Achter_Elkaar)


In [ ]:
df1.head()

In [ ]:
d = {'item_number':['KIN005','KIN005','KIN005','KIN005','KIN005','A789B','A789B','A789B','G123    H','G123H','G123H'],
'Comp_ID':['1395','1395','1395','1395','1395','7787','7787','7787','1395','1395','1395'],
'date':['2016-11-22','2016-11-21','2016-11-20','2016-11-14','2016-11-13','2016-11-22','2016-11-21','2016-11-12','2016-11-22','2016-11-21','2016-11-08']}

df = pd.DataFrame(data=d)
df.date = pd.to_datetime(df.date)
d = pd.Timedelta(1, 'D')
print (df.dtypes)


In [ ]:
df = df.sort_values(['item_number','date','Comp_ID'],ascending=False)

g = df.groupby(['Comp_ID','item_number'])
sequence2 = g['date'].apply(lambda x: x.diff().fillna(0).abs().le(d)).reset_index()
print (sequence2)

In [ ]:
sequence2.set_index('index',inplace=True)
print (sequence2)

In [67]:
d = pd.Timedelta(1, 'D')
s=df1[df1['Persoon']==905391]['Datum']
sc=s.drop_duplicates()
sc=sc.sort_values() 
sequence = sc.diff().fillna(0).abs().le(d)
print (sequence)
print (sequence * (sequence.groupby((sequence != sequence.shift()).cumsum()).cumcount() + 1))

#result = pd.merge(df1, , left_on='key', right_index=True,how='left', sort=False);

4948     True
4949    False
4950    False
4951    False
4952    False
4953    False
4954    False
4955    False
4956    False
4957     True
4958    False
4959    False
4960    False
4961     True
4962     True
4963    False
4964     True
4965    False
4966     True
4967    False
4968    False
4969     True
4970     True
4971    False
4972    False
4973    False
4974    False
4975     True
4976     True
4977    False
        ...  
5056    False
5057     True
5058    False
5059    False
5060    False
5061     True
5062     True
5063    False
5064    False
5065    False
5066    False
5067    False
5068    False
5069    False
5070    False
5071     True
5072    False
5073    False
5074     True
5075     True
5076    False
5077    False
5079    False
5080    False
5081    False
5082    False
5083     True
5084    False
5085    False
5086    False
Name: Datum, Length: 128, dtype: bool
4948    1
4949    0
4950    0
4951    0
4952    0
4953    0
4954    0
4955    0
4956    0
4957    1
4958    